In [4]:
# from model.model_type import ModelType
from service.site_scraper import SiteScraper

gsp_scraper = SiteScraper(
    name="gsp",
    base_url="https://www.gsp.ro",
    traffic=3.6,
    time_selector="p.data-author",
    block_selector="div.news-item",
    link_selector="h2 a",
    title_strategy="text"
)

digisport_scraper = SiteScraper(
    name="digisport",
    base_url="https://www.digisport.ro",
    traffic=3.3,
    time_selector="cite",
    block_selector="article.article",
    link_selector="a.article-link, h3 a.widget-latest-list-item-link",
    title_strategy="attribute",
    title_attribute="title",
    # model=ModelType.SPACY
)

fanatik_scraper = SiteScraper(
    name="fanatik",
    base_url="https://www.fanatik.ro",
    traffic=2.9,  # Estimated, adjust as needed
    time_selector="span.date",  # Often used near article blocks
    block_selector="div.article",  # Container for each article
    link_selector="h3 a",  # Headline links
    title_strategy="text",
    # model=ModelType.GPT
)

prosport_scraper = SiteScraper(
    name="prosport",
    base_url="https://www.prosport.ro",
    traffic=3.1,
    time_selector="span.date",
    block_selector="h2.article__title",  # Directly targets the title block
    link_selector="a",  # The anchor inside the h2
    title_strategy="text",
    # model=ModelType.CLAUDE
)

sites = [gsp_scraper, digisport_scraper, fanatik_scraper, prosport_scraper]
# sites = [prosport_scraper]
total_traffic = sum(site.traffic for site in sites)

In [8]:
for site in sites:
    site.compute_weight(total_traffic)
    site.scrape_recent_articles(minutes=360)
    # site.save_to_csv()
    # site.short_print()


Error, response code: 511 for Flash News: Cele mai noi informații din sport


In [9]:
for site in sites:
    site.save_to_csv()
    # site.short_print()

In [10]:
from service.story_clusterer import StoryClusterer

clusterer = StoryClusterer(sites, 360, 0.3, 0.2)
clusterer.cluster_stories()

clusterer.print_matched_clusters()


🧠 Cluster #1 — Score: 0.535

🧠 Cluster #2 — Score: 0.535

🧠 Cluster #3 — Score: 0.496

🔍 Matched Clusters Across Multiple Sites

🧠 Cluster #1 — Score: 0.535 — Sites: gsp, digisport
------------------------------------------------------------
📰 Victor Angelescu a spus clar: ce rezultat vrea la FCSB - Universitatea Craiova
🔗 https://www.digisport.ro/fotbal/liga-1/victor-angelescu-a-spus-clar-ce-rezultat-vrea-la-fcsb-universitatea-craiova-3845331
🔑 Keywords: ierarhiei, meci, primul loc, scor, urcat, victorie, înscris, învins
🏷️ Entities: Alex Dobre, Elvir Koljic, Farul, Giulești, Ionuț Vînă, Rapid, SuperLigă, dobrogeni, giuleșteni
------------------------------------------------------------
📰 Cristi Manea a spus ce rezultat vrea la FCSB - Craiova: „Sincer să vă zic...”
🔗 https://www.gsp.ro/fotbal/liga-1/cristi-manea-reactie-rapid-farul-850488.html
🔑 Keywords: fundașul dreapta, partidă, reacție, rezultat vrea, subiecte
🏷️ Entities: Cristi Manea, FCSB, Farul, Rapid, Universitatea Craiova
-

In [11]:

top_stories = clusterer.score_clusters()

for i, story in enumerate(top_stories[:5], 1):
    print(f"\n🏆 Top {i} — Score: {story['score']}")
    for article in story["articles"]:
        print(f"- {article.site}: {article.title}")



🏆 Top 1 — Score: 0.535
- digisport: Victor Angelescu a spus clar: ce rezultat vrea la FCSB - Universitatea Craiova
- gsp: Cristi Manea a spus ce rezultat vrea la FCSB - Craiova: „Sincer să vă zic...”

🏆 Top 2 — Score: 0.535
- digisport: ”Știu ce calități am”. Dennis Man a reacționat, după primul gol în tricoul lui PSV
- gsp: Dennis Man s-a descătușat după primul gol la PSV și s-a destăinuit în fața jurnaliștilor » Prin ce a trecut în primele săptămâni în Olanda

🏆 Top 3 — Score: 0.496
- digisport: Fiica ”celui mai mare interlop al țării” a văzut ce a făcut iubitul ei, a filmat totul și a pus clipul pe internet
- prosport: Fiica ”celui mai mare interlop al țării” a văzut ce a făcut iubitul ei, a filmat totul și a pus clipul pe internet
